# WISE Infilling Example

Example of using proc_continuous to infill an arbitrary masked region of an image. This examples uses a subimage from the WISE 12 um dust map (tile 115).

Requires Julia Kernel

In [ ]:
# import Pkg
# Pkg.add("FITSIO")
# Pkg.add("StatsBase")
# Pkg.add("CairoMakie")
# Pkg.add("ColorSchemes")
# Pkg.add("OffsetArrays")
# Pkg.add(url="https://github.com/andrew-saydjari/CloudClean.jl")

In [ ]:
# using Pkg
# Pkg.add("IJulia")
# using IJulia
# IJulia.installkernel("julia", "--project=@.")


    Updating registry at `C:\Users\jhoffm72\.julia\registries\General.toml`
   Resolving package versions...
   Installed libsodium_jll ─── v1.0.21+0
   Installed MbedTLS ───────── v1.1.9
   Installed ZeroMQ_jll ────── v4.3.6+0
   Installed SoftGlobalScope ─ v1.1.0
   Installed VersionParsing ── v1.3.0
   Installed ZMQ ───────────── v1.4.1
   Installed IJulia ────────── v1.29.0
   Installed Conda ─────────── v1.10.2
    Updating `C:\Users\jhoffm72\.julia\environments\v1.11\Project.toml`
  [7073ff75] + IJulia v1.29.0
    Updating `C:\Users\jhoffm72\.julia\environments\v1.11\Manifest.toml`
  [8f4d0f93] + Conda v1.10.2
  [7073ff75] + IJulia v1.29.0
  [739be429] + MbedTLS v1.1.9
  [b85f4697] + SoftGlobalScope v1.1.0
  [81def892] + VersionParsing v1.3.0
  [c2297ded] + ZMQ v1.4.1
  [8f1865be] + ZeroMQ_jll v4.3.6+0
  [a9144af2] + libsodium_jll v1.0.21+0
    Building Conda ─→ `C:\Users\jhoffm72\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\b19db3927f0db4151cb86d073689f2428e524576\b

"C:\\Users\\jhoffm72\\AppData\\Roaming\\jupyter\\kernels\\julia-1.11"

In [ ]:
# Load needed packages

using Pkg
using FITSIO
using StatsBase
using CloudClean
using CairoMakie
using ColorSchemes
using OffsetArrays

In [ ]:
#parameters. right click - format cell - mark as parameters
# This cell must be tagged with "parameters"

image_path = "default/image/path.fits"
mask_path = "default/mask/path.fits"
save_path = "default/save/path.fits"

In [ ]:
# Open the FITS file and read data and header
f = FITS(image_path)
raw_image = read(f[1])
header = read_header(f[1])
close(f)

# Make a copy of the raw image data
out_image = copy(raw_image)

# Replace NaN values with 0.0
out_image[isnan.(raw_image)] .= 0.0



CFITSIO.CFITSIOError{String}: CFITSIO has encountered an error while processing /Users/jakehoffmann/Documents/JHU_Research/FilPHANGS_Main/ngc0628_F770W/CDD/ngc0628_F770W_JWST_Emission_starsub_CDDss0016pc.fits. Error code 104: could not open the named file
Detailed error message follows: 
failed to find or open the following file: (ffopen)
/Users/jakehoffmann/Documents/JHU_Research/FilPHANGS_Main/ngc0628_F770W/CDD/ngc0
628_F770W_JWST_Emission_starsub_CDDss0016pc.fits


In [ ]:
# Mask with missing pixels
f = FITS(mask_path)
raw_mask = read(f[1])
close(f)

# Convert the raw mask data into a boolean mask (True for masked pixels, False otherwise)
bimage = raw_mask .!= 0  # Create a boolean mask, True where non-zero

# Now bimage is a boolean array with True for masked pixels and False for unmasked pixels
println("Mask shape: ", size(bimage))

In [ ]:
Np = 33
out_image[bimage].= 0
count(bimage)

In [ ]:
# Example values for cenx, ceny, and dv; adjust these based on your data
cenx = 579
ceny = 579
dv = 250

# Assuming raw_image is your image data and out_image is the image you want to display
# Compute the color range based on percentiles in a region around (cenx, ceny)
vmin, vmax = percentile(out_image[(cenx-dv):(cenx+dv), (ceny-dv):(ceny+dv)], [2, 99])

# Create a figure for plotting
fig = Figure(size=(600, 600), fontsize=18)

# Set up the axis with no ticks or labels
ax = Axis(fig[1,1], aspect=1, title="Masked Original", xticksvisible=false, yticksvisible=false, 
          xticklabelsvisible=false, yticklabelsvisible=false)

# Display the heatmap using the chosen colormap and the computed color range
h = heatmap!(ax, out_image[(cenx-dv):(cenx+dv), (ceny-dv):(ceny+dv)]',
    colormap=:rainbow_bgyr_35_85_c73_n256,
    colorrange=(vmin, vmax)
)

# Resize the figure to fit the layout
resize_to_layout!(fig)

# Show the figure
fig


In [ ]:
ndraw0 = 2
widx = 129
x_locs = [cenx]
y_locs = [ceny]
star_stats = proc_continuous(out_image,bimage,Np=Np,tilex=4,ftype=64,widx=widx,seed=2022,ndraw=ndraw0)


# Debug: Check the structure of star_stats
if isa(star_stats, Tuple)
    println("star_stats is a tuple with length: ", length(star_stats))
    for i in 1:length(star_stats)
        println("star_stats[$i] has type: ", typeof(star_stats[i]), " and size: ", size(star_stats[i]))
    end
else
    println("star_stats is not a tuple. It has type: ", typeof(star_stats))
end

In [ ]:
cenx = 579
ceny = 579
dv = 250
vmin, vmax = percentile(out_image[(cenx-dv):(cenx+dv),(ceny-dv):(ceny+dv)],[2,99])
fig = Figure(size=(600, 600), fontsize=18)
ax = Axis(fig[1,1], aspect=1, title="Mean Infill", xticksvisible=false, yticksvisible=false, 
          xticklabelsvisible=false, yticklabelsvisible=false)
h = heatmap!(ax,star_stats[1][(cenx-dv):(cenx+dv),(ceny-dv):(ceny+dv)]',
    colormap=:rainbow_bgyr_35_85_c73_n256,
    colorrange=(vmin,vmax)
)

resize_to_layout!(fig)
fig

In [ ]:
dv = 250
fig = Figure(size=(600, 600), fontsize=18)
ax = Axis(fig[1,1], aspect=1, title="Draw Infill", xticksvisible=false, yticksvisible=false, 
          xticklabelsvisible=false, yticklabelsvisible=false)
h = heatmap!(ax,star_stats[2][(cenx-dv):(cenx+dv),(ceny-dv):(ceny+dv),1]',
    colormap=:rainbow_bgyr_35_85_c73_n256,
    colorrange=(vmin,vmax)
)

# Convert to a standard Array
array_data = max.(parent(star_stats[2]), 0)  # Set negative values to zero

# Open a FITS file and write
FITS(save_path, "w") do f
    write(f, convert(Array, array_data); header = header)  # Ensure it's a proper Array
end

In [ ]:
dv = 250
fig = Figure(size=(600, 600), fontsize=18)
ax = Axis(fig[1,1], aspect=1, title="Original Image", xticksvisible=false, yticksvisible=false, 
          xticklabelsvisible=false, yticklabelsvisible=false)
h = heatmap!(ax,raw_image[(cenx-dv):(cenx+dv),(ceny-dv):(ceny+dv)]',
    colormap=:rainbow_bgyr_35_85_c73_n256,
    colorrange=(vmin,vmax)
)

resize_to_layout!(fig)
fig